# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [55]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [56]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,papatowai,-46.5619,169.4708,8.32,64,100,1.43,NZ,2024-08-02 00:34:55
1,1,kattivakkam,13.2167,80.3167,29.52,76,100,3.92,IN,2024-08-02 00:34:56
2,2,semera,11.5010,41.2010,25.11,65,100,1.28,ET,2024-08-02 00:34:57
3,3,blackmans bay,-43.0167,147.3167,9.85,78,17,0.89,AU,2024-08-02 00:34:58
4,4,barrouallie,13.2333,-61.2667,26.01,76,97,2.07,VC,2024-08-02 00:34:59


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [57]:

# Create a base map centered around the mean latitude and longitude of the cities

# Create an interactive map
map_plot = city_data_df.hvplot.points(
    x='Lng',       # Column name for longitude
    y='Lat',        # Column name for latitude
    geo=True,            # Enable geospatial plotting
    size= 'Humidity',             # Size of markers
    color='City',        # Color of markers
    hover_cols=['City'], # Columns to display on hover
    title='City Locations',
    tiles = 'OSM'
)

# Display the map
map_plot






:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [58]:
# A max temperature lower than 27 degrees but higher than 21
# Wind speed less than 4.5 m/s
# Zero cloudiness

# Narrow down cities that fit criteria and drop any results with null values
narrow_city_df = city_data_df[(city_data_df['Max Temp']>21) & (city_data_df['Max Temp']<27) & (city_data_df['Wind Speed']< 4.5)]

# Drop any rows with null values
narrow_city_df = narrow_city_df.dropna()

# Display sample data
narrow_city_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
2,2,semera,11.5010,41.2010,25.11,65,100,1.28,ET,2024-08-02 00:34:57
4,4,barrouallie,13.2333,-61.2667,26.01,76,97,2.07,VC,2024-08-02 00:34:59
6,6,broome,42.2506,-75.8330,25.43,61,63,0.45,US,2024-08-02 00:35:01
22,22,albany,42.6001,-73.9662,26.05,63,0,2.56,US,2024-08-02 00:31:16
34,34,sao gabriel da cachoeira,-0.1303,-67.0892,23.55,89,71,0.18,BR,2024-08-02 00:35:32


### Step 3: Create a new DataFrame called `hotel_df`.

In [59]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = narrow_city_df[["City", "Country", "Lat", "Lng", "Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = None

# Display sample data
hotel_df

/var/folders/sp/wj92j4n17n5267jmds8xttfc0000gn/T/ipykernel_9503/4254615678.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'] = None


,City,Country,Lat,Lng,Humidity,Hotel Name
2,semera,ET,11.5010,41.2010,65,None
4,barrouallie,VC,13.2333,-61.2667,76,None
6,broome,US,42.2506,-75.8330,61,None
22,albany,US,42.6001,-73.9662,63,None
34,sao gabriel da cachoeira,BR,-0.1303,-67.0892,89,None
...,...,...,...,...,...,...
564,busayra,JO,30.7376,35.6078,49,None
566,termoli,IT,41.9888,14.9895,57,None
570,csurgo,HU,46.2531,17.1006,94,None
576,bull savanna,JM,17.8868,-77.5902,78,None


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [60]:

# Define your search radius (in meters)
radius = 10000  

# API parameters
params = {
    'categories': 'accommodation.hotel',  # Filter to find hotels
    'radius': radius,                    # Search radius
    'apiKey': geoapify_key,    # Your Geoapify API key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Initialize a counter
counter = 0
max_results = 5  # Number of results to process

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    if counter >= max_results:
        break  # Stop if the maximum number of results has been processed

    # Get latitude and longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    # Print or use the latitude and longitude
    print(f"Index: {index}, Latitude: {lat}, Longitude: {lng}")
    
    # Update the parameters with the current city's latitude and longitude
    params["bias"] = f'proximity:{lat},{lng}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make the API request using the params dictionary
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    data = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = data["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")
    
    # Increment the counter
    counter += 1

# Display the updated DataFrame
hotel_df.head()



Starting hotel search
Index: 2, Latitude: 11.501, Longitude: 41.201
semera - nearest hotel: Barceló Aran Blu - Adults only
Index: 4, Latitude: 13.2333, Longitude: -61.2667
barrouallie - nearest hotel: Agulhas Country Lodge & Restaurant
Index: 6, Latitude: 42.2506, Longitude: -75.833
broome - nearest hotel: Keravel
Index: 22, Latitude: 42.6001, Longitude: -73.9662
albany - nearest hotel: No hotel found
Index: 34, Latitude: -0.1303, Longitude: -67.0892
sao gabriel da cachoeira - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
2,semera,ET,11.5010,41.2010,65,Barceló Aran Blu - Adults only
4,barrouallie,VC,13.2333,-61.2667,76,Agulhas Country Lodge & Restaurant
6,broome,US,42.2506,-75.8330,61,Keravel
22,albany,US,42.6001,-73.9662,63,No hotel found
34,sao gabriel da cachoeira,BR,-0.1303,-67.0892,89,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [64]:
# Create a map plot using hvplot
map_plot = hotel_df.hvplot.points(
    x='Lng',
    y='Lat',
    geo=True,
    tiles='OSM',
    size=150,
    color='City',
    hover_cols=['City']
)

# Display the map plot
map_plot



:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City)